### Attendance Streak

In [70]:
import pandas as pd
import pyodbc

In [71]:
server = '157.119.230.120,4071'    
database = 'PowerBI_LISV2'      
username = 'sa'
password = 'Y0m@Sql.!.123'

# Connection string
conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

In [72]:
x = ['AmulManagement', 'AdaniWilmerManagement', 'CholayilManagement','DanoneGTManagement','DanoneMTManagement','GCPLFoodsManagement','GCPLManagement',
     'JohnsonJohnsonManagement','ParleManagement','ColgateManagement','PidiliteManagement','PiramalManagement','ZydusMTManagement','ZydusGTManagement']

# Static query parts
designation_list = ['Merchandiser', 'Promoter', 'BA', 'BP', 'ISR', 'FloaterPromoter', 'Floater Promoter']

In [73]:
designation_str = ",".join(f"'{d}'" for d in designation_list)

In [74]:
conn = pyodbc.connect(conn_str)
Attendance_df = pd.DataFrame()

In [75]:
for value in x:
    query2 = f"""
    WITH Attendance_CTE AS (
        SELECT
            '{value}' AS client,
            M.EmpPositionCode,
            M.EmpPositionCodeStatus,
            A.UserName,
            A.FullName,
            A.Region,
            A.City,
            A.Designation,
            A.Status,
            A.[Date] AS Date2,
            M.Status AS chek,
            M.LastWorkingDate,
            CASE
                WHEN A.Status = 'Present' THEN 'P'
                WHEN A.Status = 'Absent' THEN 'A'
                WHEN A.Status = 'Leave' THEN 'L'
                WHEN A.Status = 'Compensatory Off' THEN 'CO'
                WHEN A.Status IN ('Week Off', 'WeekOff') THEN 'WO'
                WHEN A.Status = 'Meeting' THEN 'M'
                WHEN A.Status = 'Training' THEN 'T'
                WHEN A.Status IN ('National Hoilday', 'Holiday') THEN 'H'
                ELSE 'NA'
            END AS AttStatus,
            FORMAT(TRY_CONVERT(date, A.[Date], 105), 'dd') AS DayNum
        FROM PowerBI_LISV2.[{value}].ISPAttendanceMasterExport AS A
        LEFT JOIN (
            SELECT K.*
            FROM (
                SELECT *,
                       RANK() OVER (PARTITION BY UserName ORDER BY UserMasterId DESC) AS RNK
                FROM PowerBI_LISV2.[{value}].UserMaster
            ) AS K
            WHERE K.RNK = 1 
        ) AS M ON A.UserName = M.UserName
        WHERE MONTH(TRY_CONVERT(date, A.[Date], 105)) = 12
          AND A.Month = 12 AND Year =2025 AND LOWER(A.FullName) NOT LIKE '%test%'
          AND (A.UserName LIKE '%ISP%' OR A.UserName LIKE '%MR%') AND LOWER(A.UserName) NOT LIKE '%test%'
    )
    SELECT *,
           RANK() OVER (PARTITION BY UserName ORDER BY Date2 DESC) AS CURR_DAYFLAG
    FROM Attendance_CTE;
    
    
    """
    df = pd.read_sql(query2, conn)
    Attendance_df = pd.concat([Attendance_df, df], ignore_index=True)
    print(f"Running for: {value}")

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_41968\3774598972.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query2, conn)


Running for: AmulManagement
Running for: AdaniWilmerManagement
Running for: CholayilManagement
Running for: DanoneGTManagement
Running for: DanoneMTManagement
Running for: GCPLFoodsManagement
Running for: GCPLManagement
Running for: JohnsonJohnsonManagement


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_41968\3774598972.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query2, conn)


Running for: ParleManagement
Running for: ColgateManagement
Running for: PidiliteManagement
Running for: PiramalManagement
Running for: ZydusMTManagement


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_41968\3774598972.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query2, conn)


Running for: ZydusGTManagement


In [76]:
conn = pyodbc.connect(conn_str)
# Attendance_df = pd.read_sql(query2, conn)

In [77]:
Attendance_df=Attendance_df.drop_duplicates()

In [78]:
Attendance_df.head(2)

client EmpPositionCode EmpPositionCodeStatus  UserName  \
0  AmulManagement            None                  None  AMLMR027   
1  AmulManagement            None                  None  AMLMR027   

                   FullName Region    City   Designation   Status  \
0  Tejaswini Sachin Shedage   West  Mumbai  Merchandiser  Present   
1  Tejaswini Sachin Shedage   West  Mumbai  Merchandiser  Present   

                 Date2  chek LastWorkingDate AttStatus DayNum CURR_DAYFLAG  
0  28-12-2025 00:00:00  True            None         P     28            1  
1  27-12-2025 00:00:00  True            None         P     27            2

In [79]:

# --- Step 1: Prepare Data ---
df = Attendance_df.copy()
df['Date2'] = pd.to_datetime(df['Date2'], errors='coerce', dayfirst=True)
df = df.sort_values(['UserName', 'Date2'])

# --- Step 2: Normalize to one row per user-date ---
daily = (
    df.assign(AttStatus=df['AttStatus'].str.strip().str.upper())
      .groupby(['UserName', 'Date2'], as_index=False)
      .agg({'AttStatus': 'first'})  # keep one record per date
)

# --- Step 3: Define which statuses break the streak ---
presence_status = ['P', 'WO', 'M']

def calc_absent_streak(user_df):
    user_df = user_df.sort_values('Date2').reset_index(drop=True)
    streak = 0
    streaks = []
    for status in user_df['AttStatus']:
        if status == 'A':
            streak += 1
        elif status in presence_status:
            streak = 0  # break the streak
        else:
            streak = 0  # treat all other statuses as breaking it too
        streaks.append(streak)
    user_df['AbsentStreak'] = streaks
    return user_df

# --- Step 4: Apply per user ---
streak_df = daily.groupby('UserName', group_keys=False).apply(calc_absent_streak).reset_index(drop=True)

# --- Step 5: Merge back with original dataset (no duplicates) ---
final_df = df.merge(streak_df[['UserName', 'Date2', 'AbsentStreak']], on=['UserName', 'Date2'], how='left')

# --- Step 6: Add AttendanceDate (for readability) ---
# final_df['AttendanceDate'] = final_df['Date2'].dt.strftime('%d-%m-%Y')

# --- Step 7: Output ---
# final_output = final_df[['UserName', 'FullName', 'Region', 'Designation', 'AttendanceDate', 'AttStatus', 'AbsentStreak']]

# print(final_output.tail(15))


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_41968\2822677721.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  streak_df = daily.groupby('UserName', group_keys=False).apply(calc_absent_streak).reset_index(drop=True)


In [80]:
final_df=final_df.drop_duplicates()

In [81]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103911 entries, 0 to 103910
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   client                 103911 non-null  object        
 1   EmpPositionCode        48714 non-null   object        
 2   EmpPositionCodeStatus  48714 non-null   object        
 3   UserName               103911 non-null  object        
 4   FullName               103911 non-null  object        
 5   Region                 103911 non-null  object        
 6   City                   103911 non-null  object        
 7   Designation            103911 non-null  object        
 8   Status                 103182 non-null  object        
 9   Date2                  103911 non-null  datetime64[ns]
 10  chek                   96454 non-null   object        
 11  LastWorkingDate        6434 non-null    object        
 12  AttStatus              103911 non-null  obje

In [82]:
final_df.head(1)

client EmpPositionCode EmpPositionCodeStatus   UserName  \
0  AdaniWilmerManagement            None                  None  AD-ISP-01   

  FullName     Region   City Designation   Status      Date2  chek  \
0     Ekta  North One  Delhi    Promoter  Present 2025-12-01  None   

  LastWorkingDate AttStatus DayNum CURR_DAYFLAG  AbsentStreak  
0            None         P     01           28             0

In [83]:
latest_streak = (
    final_df.sort_values(['UserName', 'Date2'])
      .groupby('UserName')['AbsentStreak']
      .last()
      .reset_index()
      .rename(columns={'AbsentStreak': 'recent_absent_streak'})
)

### Supervisor name mapping

In [84]:
df_User_final =pd.DataFrame()

for value in x:
        query = f"""
        SELECT K.ISP, UM.FullName AS Supervisor
        FROM
        (SELECT DISTINCT ISP,Supervisor FROM PowerBI_LISV2.{value}.UserHierarchyListReport)AS K
        LEFT JOIN
        (SELECT K.UserName,K.FullName
            FROM (SELECT UserName,FullName,RANK() OVER (PARTITION BY UserName ORDER BY UserMasterId DESC) AS RNK
                    FROM PowerBI_LISV2.{value}.UserMaster
        			WHERE UserName LIKE '%SUP%'
                ) AS K
        WHERE K.RNK = 1) AS UM ON UM.UserName = K.Supervisor;
        """
        df_User = pd.read_sql(query, conn)
        df_User_final = pd.concat([df_User_final, df_User], ignore_index=True)

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_41968\1973400355.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_User = pd.read_sql(query, conn)
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_41968\1973400355.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_User = pd.read_sql(query, conn)
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_41968\1973400355.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_User = pd.read_sql(query, conn)


In [85]:
df_User_final.head()

ISP  Supervisor
0  AMLMR015  Pavithra T
1  AMLMR028  Pavithra T
2  AMLMR040  Pavithra T
3  AMLMR041  Pavithra T
4  AMLMR057  Pavithra T

In [86]:
final_df1 = pd.merge(final_df,df_User_final,how='left',left_on = 'UserName', right_on = 'ISP')

In [87]:
final_df1.head()

client EmpPositionCode EmpPositionCodeStatus   UserName  \
0  AdaniWilmerManagement            None                  None  AD-ISP-01   
1  AdaniWilmerManagement            None                  None  AD-ISP-01   
2  AdaniWilmerManagement            None                  None  AD-ISP-01   
3  AdaniWilmerManagement            None                  None  AD-ISP-01   
4  AdaniWilmerManagement            None                  None  AD-ISP-01   

  FullName     Region   City Designation    Status      Date2  chek  \
0     Ekta  North One  Delhi    Promoter   Present 2025-12-01  None   
1     Ekta  North One  Delhi    Promoter   Present 2025-12-02  None   
2     Ekta  North One  Delhi    Promoter  Week Off 2025-12-03  None   
3     Ekta  North One  Delhi    Promoter     Leave 2025-12-04  None   
4     Ekta  North One  Delhi    Promoter   Present 2025-12-05  None   

  LastWorkingDate AttStatus DayNum CURR_DAYFLAG  AbsentStreak        ISP  \
0            None         P     01           28             0  AD-ISP-01   
1            None         P     02           27             0  AD-ISP-01   
2            None        WO     03           26             0  AD-ISP-01   
3            None         L     04           25             0  AD-ISP-01   
4            None         P     05           24             0  AD-ISP-01   

  Supervisor  
0       None  
1       None  
2       None  
3       None  
4       None

In [88]:
# latest_streak[latest_streak['UserName']=='CLG-ISP-1122']

In [89]:
# final_df = final_df.merge(latest_streak, on='UserName', how='left')

In [90]:
# final_df[final_df['UserName']=='CLG-ISP-1122']

In [91]:
final_df1 = final_df1.drop(columns = ['ISP'])

In [92]:
final_df1.to_csv("Attendence_STREAK2.csv")